In [1]:
import json
import csv
import os
import pandas as pd

path = "C:\\github\\solliancenet\\microsoft-postgres-docs-project\\artifacts\\data"

In [2]:
def make_json(csvFilePath, jsonFilePath, keyColumn = None):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary 
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows[keyColumn]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps() 
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [3]:
#get each csv file in the directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        csvFilePath = os.path.join(path, filename)
        jsonFilePath = os.path.join(path, filename.replace(".csv", ".json"))

        key = "id"

        if (filename.startswith("calendar")):
            key = "listing_id"

        if (filename.startswith("listings")):
            key = "id"

        if (filename.startswith("reviews")):
            key = "listing_id"

        make_json(csvFilePath, jsonFilePath, key)

        continue
    else:
        continue

In [4]:
#load the json file

calendar_json = json.loads(open(f"{path}\\calendar.json", encoding='utf-8').read())
calendar_df = pd.read_csv(f"{path}\\calendar.csv", encoding='utf-8')

listings_json = json.loads(open(f"{path}\\listings.json", encoding='utf-8').read())
listings_df = pd.read_csv(f"{path}\\listings.csv", encoding='utf-8')

reviews_json = json.loads(open(f"{path}\\reviews.json", encoding='utf-8').read())
reviews_df = pd.read_csv(f"{path}\\reviews.csv", encoding='utf-8')

In [10]:
max_listings = 1000
listing_count = 0

listings_export = []

#merge the data...
for listingKey in listings_json.keys():

    if (listing_count > max_listings):
        break

    listing_count += 1

    listing = listings_json[listingKey]

    listing_calendar = calendar_df[calendar_df['listing_id'] == int(listingKey)]

    listing['calendar'] = []

    count = len(listing_calendar)

    print(f"Adding {count} calendar to listing {listingKey}")

    #add the calendar data from dataframes
    for index, row in listing_calendar.iterrows():
        item = row.to_dict()
        item.pop('listing_id', None)
        listing['calendar'].append(item)
    
    listing_reviews = reviews_df[reviews_df['listing_id'] == int(listingKey)]

    count = len(listing_reviews)

    print(f"Adding {count} reviews to listing {listingKey}")

    #add the reviews data
    listing['reviews'] = []

    for index, row in listing_reviews.iterrows():
        item = row.to_dict()
        item.pop('listing_id', None)
        listing['reviews'].append(item)

    listings_export.append(listing)

#save to a new file
with open(f"{path}\\listings_merged.json", 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(listings_export, indent=4))

Adding 365 calendar to listing 241032
Adding 207 reviews to listing 241032
Adding 365 calendar to listing 953595
Adding 43 reviews to listing 953595
Adding 365 calendar to listing 3308979
Adding 20 reviews to listing 3308979
Adding 365 calendar to listing 7421966
Adding 0 reviews to listing 7421966
Adding 365 calendar to listing 278830
Adding 38 reviews to listing 278830
Adding 365 calendar to listing 5956968
Adding 17 reviews to listing 5956968
Adding 365 calendar to listing 1909058
Adding 58 reviews to listing 1909058
Adding 365 calendar to listing 856550
Adding 173 reviews to listing 856550
Adding 365 calendar to listing 4948745
Adding 8 reviews to listing 4948745
Adding 365 calendar to listing 2493658
Adding 32 reviews to listing 2493658
Adding 365 calendar to listing 175576
Adding 181 reviews to listing 175576
Adding 365 calendar to listing 4454295
Adding 8 reviews to listing 4454295
Adding 365 calendar to listing 3883392
Adding 13 reviews to listing 3883392
Adding 365 calendar to